# GIA - Generation of NULL sequences

In [2]:
!date
!hostname

Wed Oct  9 13:15:51 PDT 2024
c4-gpudev1


In [3]:
%env CONDA_PREFIX

'/c4/home/derek/miniconda3/envs/deepripe'

In [4]:
from pathlib import Path
import pandas as pd
import json
import numpy as np
from sklearn.model_selection import GroupKFold

import pysam 
import tensorflow as tf

import seaborn as sns


#import modified basenji modules 
import sys
sys.path.append("/nowakowskilab/data1/derek/data_scSLR/prenatal_brain/deep_splicing")

from basenji_.basenji.dna_io import dna_1hot

2024-10-09 13:16:09.084071: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-09 13:16:09.435699: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [5]:
#import modified basenji modules 
sys.path.append("/nowakowskilab/data1/derek/data_scSLR/prenatal_brain/deep_splicing")
from basenji_.basenji.dna_io import dna_1hot

def region_to_mat(region):
    region_len = len(region)
    region= region.replace('i','0')
    region= region.replace('c','1')
    region= region.replace('3','2')
    region= region.replace('5','3')
    region= region.replace('N','4')
    region_code = np.zeros((4,region_len), dtype='float16')
    for i in range(region_len):
        if int(region[i]) != 4:
            region_code[int(region[i]),i] = 1
        else:
            region_code[0:4,i] = np.tile(0,4)
    return np.transpose(region_code).astype('int8')

In [6]:
pwd

'/nowakowskilab/data1/derek/data_scSLR/prenatal_brain/deep_splicing/transfer_learning'

## Load MAJIQ cassette exon annotations

In [7]:
input_dir ='/c4/home/derek/data1//derek/data_scSLR/prenatal_brain/majiq/voila_modulize_out_PSI_0.01_wConstitutive/'

cass = pd.read_csv(input_dir+'cassette.tsv',sep='\t',comment='#')

/scratch/derek/ipykernel_3726499/3976852381.py:3: DtypeWarning: Columns (15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  cass = pd.read_csv(input_dir+'cassette.tsv',sep='\t',comment='#')


In [8]:
cass.columns

Index(['module_id', 'gene_id', 'gene_name', 'seqid', 'strand', 'lsv_id',
       'event_id', 'complex', 'denovo', 'reference_exon_coord', 'spliced_with',
       'spliced_with_coord', 'junction_name', 'junction_coord', 'event_size',
       'event_non_changing', 'event_changing', 'junction_changing',
       'merged_complete_median_reads', 'merged_complete_median_psi',
       'merged_complete_var_psi'],
      dtype='object')

In [9]:
cass = cass.drop(['module_id', 'gene_name','lsv_id','event_id','complex',
      'denovo','reference_exon_coord', 'junction_name', 'junction_coord',
        'event_non_changing','event_changing','junction_changing'
          ], axis=1 )

cass = cass[cass.spliced_with == 'A']
cass = cass.drop_duplicates(subset = 'spliced_with_coord')

cass[['start','end']] = cass.spliced_with_coord.str.split('-',expand=True)

cass = cass[cass['end'].astype(int) > cass['start'].astype(int)]

In [10]:
cass['model_input'] = cass['seqid']+'_'+cass['start']+'_'+cass['end']+'_'+cass['gene_id'].str[:-3]+'_'+cass['strand']

In [11]:
in_dir = '/nowakowskilab/data1/derek/data_scSLR/prenatal_brain/deep_splicing/transfer_learning/'

null_orig = pd.read_csv(in_dir+'sig_dif_NULL_table.csv', index_col=0)
EN = pd.read_csv(in_dir+'sig_dif_EN_table_0.1_.csv', index_col=0)
RG = pd.read_csv(in_dir+'sig_dif_RG_table_0.1_.csv', index_col=0)




In [12]:
# remove exons included in the differentially spliced in EN, RG sets

cass = cass[~cass.model_input.isin(pd.concat([RG.model_input,
                                 EN.model_input,
                                 null_orig.model_input]
                               )
                     )
           ]

In [13]:
null_data = pd.DataFrame(cass.model_input).set_index('model_input')

null_data[0] = 0
null_data.index.name = None

In [14]:
null_data = null_data.sample(n = 2500)

In [15]:
NULL = null_data

In [16]:
NULL = NULL.loc[NULL.index.drop_duplicates()]

In [17]:
NULL

,0
chr9_10612398_10612453_ENSG00000153707_-,0
chr19_7630754_7630846_ENSG00000229833_+,0
chr4_98909678_98909959_ENSG00000151247_-,0
chr15_82418639_82418792_ENSG0000029094_+,0
chr1_227254064_227254155_ENSG00000143776_-,0
...,...
chr14_96593490_96593579_ENSG0000025870_+,0
chr3_10125626_10125708_ENSG0000025499_+,0
chr2_8958497_8958642_ENSG00000143797_-,0
chr10_46794727_46794915_ENSG0000029106_+,0


In [18]:
exon_info = pd.DataFrame(NULL.index)[0].str.split('_', expand=True)
exon_info = exon_info.rename(columns={0: 'chr', 1: 'start', 2: 'end',
                                      3: 'ENSG', 4: 'strand'})

In [19]:
#reduce null exons to those being > 100bp

# exon_info = exon_info[np.abs(exon_info['start'].astype(int) - exon_info['end'].astype(int) ) > 100][:1000]

# NULL = NULL.iloc[exon_info.index]

In [40]:
intron_flanking_length = 150
# exon_flanking_length = 50


exon_info['start'] = exon_info['start'].astype(int) - intron_flanking_length
exon_info['end'] = exon_info['end'].astype(int) + intron_flanking_length

In [41]:
NULL.shape

(2500, 1)

In [42]:
# NULL = NULL.iloc[exon_info.index]

In [46]:
exon_info_set = exon_info.copy()

num_targets = 2
max_length = 600
region_length = 1000
num_layers=5

extra = int((region_length - max_length)/8)


In [47]:
extra

50

In [48]:
                    # define ranges for all regions of interest and pad if shorter than intron_flanking_length
                    


                    # # upstream_intron 
                    
                    # upstream_intron = seq_DNA[0:intron_flanking_length]
                    
                    # upstream_intron_region = ('i'*extra)+('i'*intron_flanking_length)+('c'*extra)
                    
                    # #exon_start
                    
                    # exon_coord = len(seq_DNA) // 2
                    
                    # if exon_coord <= (exon_flanking_length * 2):
                    
                    #     exon_start = seq_DNA[intron_flanking_length:exon_coord]
                    
                    #     padding_start = exon_flanking_length - len(exon_start)

                    #     exon_start = exon_start + padding_start*'N'

                        
                    # else: 
                        
                    #     exon_start =  seq_DNA[intron_flanking_length:(intron_flanking_length + exon_flanking_length)]


                    
                    
                    # exon_start_region = ('i'*extra)+''.join(['c' if x != 'N' else 'N' for x in exon_start])+('N'*extra)
                    
                    # # downstream_intron
                    
                    # downstream_intron = seq_DNA[(len_DNA-intron_flanking_length):len_DNA]
                    
                    # downstream_intron_region = ('c'*extra)+('i'*intron_flanking_length)+('i'*extra)

                    # # exon_end
                    
                    # exon_coord = len(seq_DNA) // 2 
                    
                    # if exon_coord <= (exon_flanking_length * 2): 
                    
                    #     exon_end = seq_DNA[exon_coord:(len_DNA-intron_flanking_length)]
                        
                    #     padding_end = exon_flanking_length - len(exon_end)

                    #     exon_end = padding_end*'N' + exon_end

        
                        
                    # else:
                        
                    #     exon_end = seq_DNA[(len_DNA-(intron_flanking_length + exon_flanking_length)):(len_DNA-intron_flanking_length)]












                    
                    # upstream_intron 

In [49]:
max_length

600

In [54]:
genes = exon_info['ENSG']
num_fold = 1
file_folds = None
peaks=0
prefix='ALL_CASSETTE_PARCLIP_tfrecord'


file_fasta = "/c4/home/derek/data1/derek/reference/human_hg38_reference/refdata-gex-GRCh38-2020-A/fasta/genome.fa"
general_out_dir=f'./data_out/{prefix}/'

In [55]:
def create_tfrecords_deepripe_null(PSI, exon_info, train_idxs, 
                  fold, general_out_dir, peaks,
                     max_length, num_layers):
    
    fold_dir = general_out_dir + '/fold' + str(num_fold)
    tfr_dir = fold_dir + '/tfrecords' 
    Path(fold_dir).mkdir(parents=True, exist_ok=True)
    Path(tfr_dir).mkdir(parents=True, exist_ok=True)

    fold_indexes = [train_idxs]
    
    # Write the file genes.csv
    split = np.zeros((len(exon_info),1), dtype='<U5')
    split[train_idxs] = 'train'
    target = PSI.values
    genes = pd.DataFrame(np.hstack((split,target)), 
                         index=PSI.index, columns=np.hstack((['split'], PSI.columns)))
    
    genes = genes[genes['split'] != '']
    genes.to_csv(fold_dir + '/genes.csv')
    
    # Options TFR writer
    tf_opts = tf.io.TFRecordOptions(compression_type='ZLIB')
    seqs_per_tfr = 256
    fold_labels = ['train']

    
    # open FASTA
    fasta_open = pysam.Fastafile(file_fasta)
    
    def rc(seq):
        return seq.translate(str.maketrans("ATCGatcg","TAGCtagc"))[::-1]
    
    def _bytes_feature(value):
        return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))
    
    # Write the TFRecords
    num_folds = 1 #train-val-test
    
    for fi in range(num_folds):
        exon_ID_set = PSI.index[fold_indexes[fi]]
        exon_info_set = exon_info.iloc[fold_indexes[fi]]
        PSI_val_set = PSI.iloc[fold_indexes[fi]]
        
        num_set = exon_ID_set.shape[0]
        
        num_set_tfrs = int(np.ceil(num_set / seqs_per_tfr)) 
        # print(num_set_tfrs)
        
        # gene sequence index
        si = 0
        
        for tfr_i in range(num_set_tfrs):
            # Create the file e.g 'tfr_records/test-0.tfr'
            tfr_file = '%s/%s-%d.tfr' % (tfr_dir, fold_labels[fi], tfr_i)
            # print(tfr_file)
            with tf.io.TFRecordWriter(tfr_file, tf_opts) as writer:
                # TFR index
                ti = 0
                
                # This is to make sure that the max. genes per file stays below 256
                # And that for the last batch we stop on time (si < num_set)
                while ti < seqs_per_tfr and si < num_set:
                    # Get the genes that should be in this set
                    
                    seq_chrm = exon_info_set['chr'].iloc[si]
                    seq_start = int(exon_info_set['start'].iloc[si])
                    seq_end = int(exon_info_set['end'].iloc[si])+1 #Because fasta.fetch doesn't include end bp
                    seq_strand = exon_info_set['strand'].iloc[si]
                    gene = exon_info_set['ENSG'].iloc[si]
                    
                    #start
                    seq_start_ = seq_start
                    #end
                    seq_end_ = seq_end
                    
                    
                    #get exon's range
                    seq_DNA = fasta_open.fetch(seq_chrm, seq_start_, seq_end_)
                    
                    #and its length
                    len_DNA = len(seq_DNA)
                    
                    # get positions of splice sites
                    splicing_ind = np.array([intron_flanking_length, max_length - intron_flanking_length], dtype=np.int64)

                    
                    # orient
                    if seq_strand == '-':
                        seq_DNA = rc(seq_DNA)
                        
                                  
                    upstream_intron = seq_DNA[0:intron_flanking_length]
                    
                    upstream_intron_region = ('i'*extra)+('i'*intron_flanking_length)+('c'*extra)
                    
                    #exon_start
                    
                    exon_coord = len(seq_DNA) // 2
                    
                    if exon_coord < (intron_flanking_length * 2):
                    
                        exon_start = seq_DNA[intron_flanking_length:exon_coord]
                    
                        padding_start = intron_flanking_length - len(exon_start)

                        exon_start = exon_start + padding_start*'N'
                        
                    else: 
                        
                        exon_start =  seq_DNA[intron_flanking_length:(intron_flanking_length * 2)]
                    
                    exon_start_region = ('i'*extra)+''.join(['c' if x != 'N' else 'N' for x in exon_start])+('N'*extra)
                    
                    # downstream_intron
                    
                    downstream_intron = seq_DNA[(len_DNA-intron_flanking_length):len_DNA]
                    
                    downstream_intron_region = ('c'*extra)+('i'*intron_flanking_length)+('i'*extra)

                    # exon_end
                    
                    exon_coord = len(seq_DNA) // 2 
                    
                    if exon_coord < (intron_flanking_length * 2):
                    
                        exon_end = seq_DNA[exon_coord:(len_DNA-intron_flanking_length)]
                        
                        padding_end = intron_flanking_length - len(exon_end)
                        
                        exon_end = padding_end*'N' + exon_end
                        
                    else:
                        
                        exon_end = seq_DNA[(len_DNA-(intron_flanking_length * 2)):(len_DNA-intron_flanking_length)]

                    
                    exon_end_region = ('N'*extra)+''.join(['c' if x != 'N' else 'N' for x in exon_end])+('i'*extra)
                        
                        
                    seq_DNA = (upstream_intron + exon_start + exon_end + downstream_intron)
                      
                    new_LEN = len(seq_DNA)
                    if (new_LEN != max_length):
                        print(exon_info_set.iloc[si])
                        print(new_LEN)
                        
                    assert(new_LEN == max_length)
                    
                    seq_region = upstream_intron_region + exon_start_region + exon_end_region + downstream_intron_region
                    assert(len(seq_region) == region_length)
                
                  
                    
                    # one hot code
                    seq_1hot = dna_1hot(seq_DNA)
                    seq_len = np.array(len(seq_DNA), dtype=np.int64)
                    
                    # splicing
                    splicing = np.zeros((new_LEN ,1), dtype=np.int8)
                    splicing[splicing_ind] = 1
                    
                    # region 
                    region_1hot = region_to_mat(seq_region)
           
                    # get targets
                    targets = PSI_val_set.iloc[si].values
                    targets = targets.reshape((1,-1)).astype('float64')
                    
            ### generate example ###
                    example = tf.train.Example(features=tf.train.Features(feature={
                           
                            'length': _bytes_feature(seq_len.flatten().tobytes()),
                            'sequence': _bytes_feature(seq_1hot.flatten().tobytes()),
                            'target': _bytes_feature(targets.flatten().tobytes()),
                            'splicing': _bytes_feature(splicing.flatten().tobytes()), 
                            'region': _bytes_feature(region_1hot.flatten().tobytes())
                        }))
    
                    # write
                    writer.write(example.SerializeToString())
    
                    # advance indexes
                    ti += 1
                    si += 1
    
    
    fasta_open.close()
    
    
    # Write statistics.json
    
    stats_dict = {}
    stats_dict['num_targets'] = num_targets
    stats_dict['seq_length'] = max_length
    stats_dict['target_length'] = 4
    
    for fi in range(num_folds):
        stats_dict['%s_seqs' % fold_labels[fi]] = len(fold_indexes[fi])
    
    with open('%s/statistics.json' % fold_dir, 'w') as stats_json_out:
        json.dump(stats_dict, stats_json_out, indent=4)
    
    
    # Copy the params.json
    train_dict = {}
    train_dict['batch_size'] = 64
    train_dict['optimizer'] = 'adam'
    train_dict['loss'] = 'bce' #'mse'
    train_dict['learning_rate'] = 0.0001
    train_dict['adam_beta1'] = 0.90
    train_dict['adam_beta2'] = 0.998
    train_dict['global_clipnorm'] = 0.5
    train_dict['train_epochs_min'] = 100
    train_dict['train_epochs_max'] = 1000
    train_dict['patience'] = 100
    
    model_dict = {}
    model_dict['activation'] = 'relu'
    model_dict['spline'] = False
    model_dict['rnn_type'] = 'gru'
    model_dict['final_activation'] = 'relu'
    model_dict['residual'] = False
    model_dict['seq_length'] = max_length

    model_dict['seq_depth'] = 4 
    model_dict['augment_shift'] = 0
    model_dict['num_targets'] = num_targets
    model_dict['heads'] = 1
    model_dict['filters'] = 186
    model_dict['kernel_size'] = 6 #5
    model_dict['dropout'] = 0.3 
    model_dict['l2_scale'] = 0.001
    model_dict['ln_epsilon'] = 0.007
    model_dict['bn_momentum'] = 0.90
    
    params_dict = {}
    params_dict['train'] = train_dict
    params_dict['model'] = model_dict
    
    with open('%s/params.json' % fold_dir, 'w') as params_json_out:
        json.dump(params_dict, params_json_out, indent=4)

In [56]:
general_out_dir

'./data_out/ALL_CASSETTE_PARCLIP_tfrecord/'

In [57]:
create_tfrecords_deepripe_null(NULL,
                        exon_info, 
                        np.arange(0,len(NULL)),       
                        num_fold, 
                       general_out_dir,
                        peaks,
                       max_length, 
                        num_layers
                        )